<a href="https://colab.research.google.com/github/mankidal19/multi-agent/blob/main/ai-agents/agent-development-kit/orchestrating_multi_agent_systems.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Copyright 2025 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Orchestrating Multi-Agent Systems with Google Agent Development Kit

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/iamthuya/google-cloud-workshops/blob/main/ai-agents/agent-development-kit/orchestrating_multi_agent_systems.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo"><br> Run in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2Fiamthuya%2Fgoogle-cloud-workshops%2Fmain%2Fai-agents%2Fagent-development-kit%2Forchestrating_multi_agent_systems.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo"><br> Run in Colab Enterprise
    </a>
  </td>      
  <td style="text-align: center">
    <a href="https://github.com/iamthuya/google-cloud-workshops/blob/main/ai-agents/agent-development-kit/orchestrating_multi_agent_systems.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/iamthuya/google-cloud-workshops/blob/main/ai-agents/agent-development-kit/orchestrating_multi-agent_systems.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> Open in Vertex AI Workbench
    </a>
  </td>
</table>

| | |
|-|-|
|Author(s) | [Thu Ya Kyaw](https://github.com/iamthuya) |

## Environment Setup

### Install Necessary Libraries

Here you will install required Python packages for this lab.

In [2]:
%pip install -U -q google-adk

In [3]:
# Check ADK version
from google import adk

print(adk.__version__)

1.4.2


### Restart current runtime

To use the newly installed packages in this Jupyter runtime, you must restart the runtime. You can do this by running the cell below, which will restart the current kernel.

Don't worry if you see a notification message like `Your session crashed for an unknown reason.` This is expected as you are shutting down the kernel from the same instance using code.


In [4]:
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

<div class="alert alert-block alert-warning">
<b>⚠️ The kernel is going to restart. Please wait until it is finished before continuing to the next step. ⚠️</b>
</div>

### Import Required Libraries

In [4]:
import os
import asyncio

from google.adk.agents import LlmAgent, SequentialAgent, ParallelAgent, LoopAgent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search, ToolContext
from google.genai import types

from IPython.display import Markdown, display

### Define Environment Variables

In [5]:
from google.colab import userdata

os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "False"
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

## Sequential Agents

In this section, you'll create an agent that researches a topic, then writes in detail about it, and finally edits the output. These steps must happen in order: research first, then writing, and then editing, because you can't write without research, and you can't edit without a written piece.

In [6]:
# Define variables
GEMINI_MODEL_NAME = "gemini-2.5-flash-preview-04-17"
APP_NAME = "sequential_app"
USER_ID = "sequential_user"
SESSION_ID = "sequential_session"

#### Initialize the agents and pipeline

In [7]:
# Define respective agents and pipeline
researcher = LlmAgent(
    name="ResearcherAgent",
    model=GEMINI_MODEL_NAME,
    instruction="Your task is to use 'google_search' tool to find all the relavent information about a given topic and write a comprehensive report about that topic.",
    tools=[google_search],
    description="Researches about a topic",
    output_key="research_results"
)

writer = LlmAgent(
    name="WriterAgent",
    model=GEMINI_MODEL_NAME,
    instruction="Your task is to write a comprehensive report using this information: {research_results}. Make sure to expend on all the topics from the provided information.",
    description="Write about a comprehensive report based on the provided information",
    output_key="comprehensive_report"
)

editor = LlmAgent(
    name="EditorAgent",
    model=GEMINI_MODEL_NAME,
    instruction="Your task is to edit a report: {comprehensive_report}. Perform a quality check on the text and improve it if necessary.",
    description="Perform quality check on a written report",
)

sequential_pipeline = SequentialAgent(
    name="ContentPipelineAgent",
    sub_agents=[researcher, writer, editor],
    description="Executes a sequence of researching, writing, and editing.",
)

### Initialize Session & Runner

In [8]:
# Initialize Session
session_service = InMemorySessionService()

session = await session_service.create_session(
    app_name=APP_NAME,
    user_id=USER_ID,
    session_id=SESSION_ID
)

# Initialize Runner
sequential_runner = Runner(
    agent=sequential_pipeline,
    app_name=APP_NAME,
    session_service=session_service,
)

### Define a function to interact with Agent

In [9]:
# Define a function to interact with the agent
async def run_agent(runner, user_id, session_id, user_query):
    user_message = types.Content(
        role="user",
        parts=[types.Part(text=user_query)]
    )

    events = runner.run_async(
        user_id=user_id,
        session_id=session_id,
        new_message=user_message
    )

    output = {}
    async for event in events:
        if event.is_final_response():
            author = event.author
            report = event.content.parts[0].text
            output[author] = report

            display(Markdown(f"## {author}"))
            display(Markdown(f"{report}"))

    return output

In [10]:
topic = "Write about the current state of generative ai"
output = await run_agent(sequential_runner, USER_ID, SESSION_ID, topic)
report = output['EditorAgent']

## ResearcherAgent

Generative AI is a rapidly evolving field experiencing significant growth and widespread adoption across various industries. Unlike traditional AI that primarily focuses on analysis and prediction, generative AI is capable of creating new content, including text, images, audio, code, simulations, and videos. This capability has moved generative AI from an emerging technology to a key focus for businesses and individuals alike.

**Current State and Advancements:**

The use of generative AI has seen a dramatic increase in recent years. A McKinsey survey indicates that the use of generative AI in organizations jumped from 33% in 2023 to 71% in 2024. This surge is attributed to advancements in deep learning algorithms, transformer architectures, and the increased availability of cloud computing resources. The landscape of generative AI models has exploded, with hundreds of models now available, including highly specialized ones. Beyond large language models (LLMs), there's a growing prominence of large multimodal models (LMMs) that can process and generate content from text, sounds, pictures, videos, and music, making AI outputs multi-sensory. Small language models (SLMs) are also gaining traction due to their faster responsiveness and lower resource requirements. Techniques like Retrieval-Augmented Generation (RAG) are being used to improve the accuracy and timeliness of generative AI outputs by allowing models to access and incorporate up-to-date external information. The development of autonomous AI agents capable of performing tasks and complex problem-solving is another significant trend.

**Applications Across Industries:**

Generative AI is finding practical applications in numerous sectors:

*   **Content Creation:** Generating text for marketing, articles, and code; creating images, audio, and video content.
*   **Marketing and Sales:** Creating personalized marketing content, product descriptions, and engaging customer interactions.
*   **Software Engineering:** Assisting with code creation, completion, and refactoring; automating development and testing strategies.
*   **Healthcare and Pharmaceuticals:** Speeding up drug discovery and clinical trials, predicting protein structures, and crafting patient-specific treatment plans.
*   **Manufacturing:** Improving quality control and automating processes.
*   **Education:** Creating personalized learning experiences and AI-driven instruction; explaining concepts, summarizing articles, and suggesting research ideas.
*   **E-commerce:** Enhancing product listings, customer communication, and generating marketing content.
*   **Legal Services:** Simplifying and facilitating the drafting of legal documents.
*   **Design:** Automating design efforts for websites and mobile apps, assisting with product and graphic design.
*   **Customer Service:** Automating customer service tasks and providing personalized responses through conversational AI.

Organizations are reporting value creation from generative AI, including increased revenue and meaningful cost reductions within business units using these technologies.

**Market Trends and Investment:**

The generative AI market is experiencing explosive growth. Valued at $21.3 billion in 2024, it is projected to reach figures around $1005.07 billion by 2034, with a significant CAGR. North America, particularly the U.S., currently dominates the market due to leadership in AI innovation, the presence of major tech companies, and robust venture capital investment. Investment in AI, generally, and generative AI specifically, has skyrocketed, indicating a shift from experimental phases to widespread implementation. Companies are prioritizing investments with clear ROI and business impact, focusing on targeted solutions rather than generic applications.

**Challenges and Limitations:**

Despite the rapid advancements and adoption, generative AI faces several significant challenges:

*   **Data Quality and Consistency:** Generative AI models require high-quality, consistent data for optimal performance. Issues like missing values, incorrect entries, and formatting inconsistencies pose significant hurdles.
*   **Data Privacy and Security:** Training models on massive datasets, which may include personal information, raises concerns about data privacy and security.
*   **Bias and Fairness:** Generative AI can amplify existing biases present in training data, potentially leading to biased or discriminatory outputs.
*   **Misinformation and Harmful Content:** The ability to generate realistic-seeming fake content, such as deepfakes, poses risks of spreading misinformation and harmful content.
*   **Copyright and Intellectual Property:** The use of copyrighted material in training data and the generation of content that may resemble existing works raise complex legal and ethical questions regarding authorship and ownership.
*   **Hallucinations and Accuracy:** Generative AI models can sometimes produce incorrect or nonsensical information presented as fact (hallucinations), which is particularly risky in fields requiring high accuracy.
*   **High Computational Costs:** Training and running large generative AI models require significant computing power and resources, making them expensive and raising concerns about environmental impact.
*   **Lack of Transparency and Explainability:** Understanding how generative AI models arrive at their outputs can be challenging, leading to a lack of transparency and making it difficult to identify and address issues like bias.
*   **Job Displacement:** The increasing automation capabilities of generative AI raise concerns about potential job displacement in various sectors.
*   **Integration Complexity:** Integrating generative AI solutions into existing organizational workflows can be complex and requires managing technological infrastructure and specialized skills.
*   **Data Limitations:** At some point, models built on existing human knowledge may exhaust the available high-quality data, potentially impacting performance.

**Ethical Considerations and Governance:**

Ethical considerations are paramount in the development and deployment of generative AI. Key ethical concerns include transparency (indicating when content is AI-generated), fairness and non-discrimination, accountability and human oversight, and data privacy and informed consent. Addressing these concerns requires strong safeguards, considering the environmental impact, and empowering users with control over their data and interactions with AI. Implementing AI governance frameworks and security tools can help mitigate risks related to authorship, bias, and data privacy. The ethical landscape is complex, with ongoing discussions around potential harms from misaligned AI systems.

**Future Outlook:**

Generative AI is expected to continue its trajectory of growth and integration. Trends point towards more autonomous AI agents, continued investment focused on customization and ROI, and opportunities for agile startups in the field. The technology is expected to become more embedded in enterprise operations, with businesses exploring dynamic and robust AI applications in the cloud. The demand for high-quality training data will remain crucial, with advancements in annotation techniques, synthetic data generation, and human-in-the-loop oversight playing important roles. While challenges remain, the potential of generative AI to transform industries and redefine how we work and interact with technology is widely acknowledged.

## WriterAgent

## Comprehensive Report on the Current State of Generative AI

**Introduction**

Generative AI represents a significant evolution within the field of artificial intelligence. Unlike traditional AI systems primarily focused on analyzing existing data and making predictions, generative AI possesses the capability to create entirely new content. This includes a wide range of outputs such as text, images, audio, code, simulations, and videos. This creative ability has rapidly elevated generative AI from an emerging technology to a central focus for both businesses and individuals across numerous sectors.

**Current State and Advancements**

The adoption and use of generative AI have witnessed dramatic growth in recent years. According to a McKinsey survey, the implementation of generative AI within organizations surged from 33% in 2023 to 71% in 2024, indicating a rapid integration into operational workflows. This accelerated progress is largely attributed to key technological advancements, including sophisticated deep learning algorithms, the development of transformer architectures, and the increased availability of powerful cloud computing resources.

The landscape of generative AI models has expanded significantly, with hundreds of models now available, many of which are highly specialized for particular tasks or domains. Beyond the prominent Large Language Models (LLMs) known for text generation, there is a growing focus on Large Multimodal Models (LMMs). These models are capable of processing and generating content across multiple modalities, including text, sounds, pictures, videos, and music, leading to more multi-sensory AI outputs. Simultaneously, Small Language Models (SLMs) are gaining traction due to their advantages in faster response times and lower computational resource requirements.

Innovations in techniques are also improving the practical application of generative AI. Retrieval-Augmented Generation (RAG), for instance, is being increasingly used to enhance the accuracy and timeliness of generated outputs by enabling models to access and incorporate relevant, up-to-date external information. Another significant trend is the development of autonomous AI agents, which are designed to perform complex tasks and engage in problem-solving independently.

**Applications Across Industries**

Generative AI is currently being deployed to create value and drive innovation across a broad spectrum of industries:

*   **Content Creation:** Automating the generation of diverse content types, including text for marketing materials, articles, and code, as well as creating images, audio, and video.
*   **Marketing and Sales:** Enabling the creation of highly personalized marketing content, compelling product descriptions, and more engaging customer interactions.
*   **Software Engineering:** Providing assistance with coding tasks, including code creation, completion, and refactoring, while also automating aspects of development and testing strategies.
*   **Healthcare and Pharmaceuticals:** Accelerating critical processes such as drug discovery and clinical trials, aiding in predicting protein structures, and helping to craft patient-specific treatment plans.
*   **Manufacturing:** Contributing to improvements in quality control procedures and automating various manufacturing processes.
*   **Education:** Facilitating the creation of personalized learning experiences, powering AI-driven instructional tools, assisting in explaining complex concepts, summarizing articles, and suggesting research ideas.
*   **E-commerce:** Enhancing product listings, improving customer communication, and generating marketing content tailored for online retail.
*   **Legal Services:** Streamlining and simplifying the process of drafting various legal documents.
*   **Design:** Automating design efforts for digital interfaces like websites and mobile apps, and providing assistance in product and graphic design.
*   **Customer Service:** Automating customer service tasks and providing personalized responses through conversational AI interfaces.

Organizations utilizing generative AI are reporting tangible benefits, including increased revenue streams and significant cost reductions within the business units where these technologies are implemented.

**Market Trends and Investment**

The generative AI market is experiencing a period of explosive growth. Currently valued at $21.3 billion in 2024, projections indicate a substantial increase to figures around $1005.07 billion by 2034, reflecting a very significant Compound Annual Growth Rate (CAGR). North America, particularly the United States, holds a dominant position in the market, largely due to its leadership in AI innovation, the presence of major technology companies driving development, and robust venture capital investment.

Investment in AI generally, and generative AI specifically, has seen a dramatic surge. This indicates a clear shift from purely experimental phases towards widespread implementation and practical application. Companies are increasingly prioritizing investments that demonstrate clear Return on Investment (ROI) and verifiable business impact, focusing on developing or adopting targeted solutions rather than broad, generic applications.

**Challenges and Limitations**

Despite rapid advancements and growing adoption, generative AI is still navigating several significant challenges and limitations:

*   **Data Quality and Consistency:** Optimal performance relies heavily on high-quality, consistent training data. Issues such as missing values, incorrect entries, and formatting inconsistencies within datasets pose substantial hurdles.
*   **Data Privacy and Security:** Training models on vast datasets, which often contain personal information, raises serious concerns regarding data privacy and overall security.
*   **Bias and Fairness:** Generative AI models can inadvertently learn and amplify biases present in their training data, potentially resulting in outputs that are biased or discriminatory.
*   **Misinformation and Harmful Content:** The technology's capacity to generate highly realistic fake content, such as deepfakes, presents risks related to the widespread dissemination of misinformation and harmful material.
*   **Copyright and Intellectual Property:** The use of copyrighted data in training models and the potential for generated content to closely resemble existing works raise complex legal and ethical questions surrounding authorship and ownership rights.
*   **Hallucinations and Accuracy:** A notable limitation is the potential for generative AI models to produce incorrect or nonsensical information presented as factual ("hallucinations"), posing particular risks in applications requiring high degrees of accuracy.
*   **High Computational Costs:** Training and operating large-scale generative AI models require substantial computing power and resources, making them expensive and raising concerns about their environmental footprint.
*   **Lack of Transparency and Explainability:** Understanding the precise reasoning process or how generative AI models arrive at specific outputs can be challenging, leading to a lack of transparency that makes it difficult to identify and address issues like bias or errors.
*   **Job Displacement:** The increasing automation capabilities offered by generative AI applications raise concerns about the potential for job displacement across various industries.
*   **Integration Complexity:** Integrating generative AI solutions into existing organizational workflows and legacy systems can be complex, requiring significant effort in managing technological infrastructure and developing or acquiring specialized skills.
*   **Data Limitations:** There is a potential future challenge where models built predominantly on existing human knowledge might eventually exhaust the supply of available, high-quality training data, which could potentially impact future performance improvements.

**Ethical Considerations and Governance**

Ethical considerations are fundamental to the responsible development and deployment of generative AI. Key ethical concerns receiving significant attention include ensuring transparency (clearly indicating when content is AI-generated), promoting fairness and preventing discrimination, establishing accountability mechanisms, ensuring adequate human oversight, and safeguarding data privacy through informed consent.

Addressing these concerns necessitates the implementation of strong safeguards, careful consideration of the environmental impact of the technology, and empowering users with control over their data and interactions with AI systems. The establishment of robust AI governance frameworks and the deployment of security tools are crucial steps to help mitigate risks related to issues like authorship, bias, and data privacy. The ethical landscape surrounding generative AI remains complex, with ongoing discussions focusing on potential harms, particularly those stemming from misaligned AI systems.

**Future Outlook**

Generative AI is widely expected to continue its current trajectory of rapid growth and increasing integration across industries. Future trends point towards the development and deployment of more autonomous AI agents capable of complex tasks. Continued investment is anticipated, with a strong focus on developing customized solutions that deliver clear ROI. This environment also presents opportunities for agile startups to innovate within the field.

The technology is projected to become increasingly embedded within enterprise operations, with businesses exploring dynamic and robust AI applications, often leveraging cloud infrastructure. The demand for high-quality training data will remain a crucial factor, driving advancements in data annotation techniques, the generation of synthetic data, and the implementation of human-in-the-loop oversight mechanisms. While the challenges outlined above persist, the potential of generative AI to fundamentally transform industries, redefine work processes, and change how humans interact with technology is broadly acknowledged and anticipated.

## EditorAgent

Okay, I have reviewed the provided report titled "Comprehensive Report on the Current State of Generative AI".

The report is well-structured, covering the introduction, current state and advancements, applications, market trends, challenges, ethical considerations, and future outlook. The content is informative and logically flows from one section to the next. It effectively incorporates the key points identified by the ResearcherAgent regarding the rapid growth, technological drivers, model types, techniques, diverse applications, market dynamics, and significant challenges/ethical issues.

The language is professional and clear. The statistics provided (McKinsey survey, market valuation) are presented clearly. The use of bullet points in the applications and challenges sections enhances readability.

Overall, the report is comprehensive and of high quality. There are no significant grammatical errors, typos, or structural issues that require correction. The writing is concise and effectively communicates the current state and future potential of generative AI while also highlighting important considerations and challenges.

**Conclusion of Quality Check:**

The report is well-written, accurate based on the provided information, and effectively structured. It meets the standard for a comprehensive report on the topic. No significant edits or improvements are necessary.

## Parallel Agents

Sometimes, running agents in parallel is desirable, particularly for tasks like translation. The Agent Developer Kit supports such parallel orchestration. In this session, you will create a parallel pipeline designed to translate a report into several languages simultaneously.

In [11]:
# Define variables
APP_NAME = "parallel_app"
USER_ID = "parallel_user"
SESSION_ID = "parallel_session"

#### Initialize the agents and pipeline

In [12]:
# Define respective agents and pipeline
bahasa_translator = LlmAgent(
    name="BahasaAgent",
    model=GEMINI_MODEL_NAME,
    instruction="Your task is to translate the given report into Bahasa lanaguage",
    description="Translate a given report to Bahasa",
    output_key="bahasa_version"
)

thai_translator = LlmAgent(
    name="ThaiAgent",
    model=GEMINI_MODEL_NAME,
    instruction="Your task is to translate the given report into Thai lanaguage",
    description="Translate a given report to Thai",
    output_key="thai_version"
)

vietnamese_translator = LlmAgent(
    name="VietnameseAgent",
    model=GEMINI_MODEL_NAME,
    instruction="Your task is to translate the given report into Vietnamese lanaguage",
    description="Translate a given report to Vietnamese",
    output_key="vietnam_version"
)

parallel_pipeline = ParallelAgent(
    name="TranslationPipelineAgent",
    sub_agents=[bahasa_translator, thai_translator, vietnamese_translator],
    description="Runs multiple tranlsation agents in parallel to translate the report.",
)

#### Initialize Session & Runner

In [14]:
# Initialize Session
session_service = InMemorySessionService()

session = await session_service.create_session(
    app_name=APP_NAME,
    user_id=USER_ID,
    session_id=SESSION_ID
)

# Initialize Runner
parallel_runner = Runner(
    agent=parallel_pipeline,
    app_name=APP_NAME,
    session_service=session_service,
)

In [15]:
prompt = f"translate this report: {report}"
output = await run_agent(parallel_runner, USER_ID, SESSION_ID, prompt)

## BahasaAgent

Tentu, berikut adalah terjemahan dari laporan tersebut ke dalam Bahasa Indonesia:

Baik, saya telah meninjau laporan yang disediakan berjudul "Laporan Komprehensif tentang Keadaan Kecerdasan Buatan Generatif Saat Ini".

Laporan ini terstruktur dengan baik, mencakup pendahuluan, keadaan dan kemajuan terkini, aplikasi, tren pasar, tantangan, pertimbangan etis, dan prospek masa depan. Isinya informatif dan mengalir secara logis dari satu bagian ke bagian berikutnya. Laporan ini secara efektif menggabungkan poin-poin penting yang diidentifikasi oleh ResearcherAgent mengenai pertumbuhan yang cepat, pendorong teknologi, jenis model, teknik, berbagai aplikasi, dinamika pasar, serta tantangan/masalah etis yang signifikan.

Bahasa yang digunakan profesional dan jelas. Statistik yang disediakan (survei McKinsey, valuasi pasar) disajikan dengan jelas. Penggunaan poin-poin dalam bagian aplikasi dan tantangan meningkatkan keterbacaan.

Secara keseluruhan, laporan ini komprehensif dan berkualitas tinggi. Tidak ada kesalahan tata bahasa, salah ketik (typo), atau masalah struktural signifikan yang memerlukan koreksi. Tulisannya ringkas dan secara efektif mengkomunikasikan keadaan terkini dan potensi masa depan AI generatif sambil juga menyoroti pertimbangan dan tantangan penting.

**Kesimpulan Pemeriksaan Kualitas:**

Laporan ini ditulis dengan baik, akurat berdasarkan informasi yang disediakan, dan terstruktur secara efektif. Laporan ini memenuhi standar untuk laporan komprehensif mengenai topik tersebut. Tidak diperlukan editan atau perbaikan yang signifikan.

## ThaiAgent

ตกลง, ฉันได้ทบทวนรายงานที่ได้รับเรื่อง "รายงานฉบับสมบูรณ์เกี่ยวกับสถานะปัจจุบันของปัญญาประดิษฐ์เชิงสร้างสรรค์" แล้ว

รายงานนี้มีโครงสร้างที่ดี ครอบคลุมตั้งแต่บทนำ สถานะปัจจุบันและความก้าวหน้า การประยุกต์ใช้ แนวโน้มตลาด ความท้าทาย ข้อควรพิจารณาทางจริยธรรม และแนวโน้มในอนาคต เนื้อหามีข้อมูลครบถ้วนและมีความเชื่อมโยงอย่างเป็นระบบจากหัวข้อหนึ่งไปยังอีกหัวข้อหนึ่ง รายงานนี้ได้รวมประเด็นสำคัญที่ระบุโดย ResearcherAgent ได้อย่างมีประสิทธิภาพ เกี่ยวกับการเติบโตอย่างรวดเร็ว ปัจจัยขับเคลื่อนทางเทคโนโลยี ประเภทของโมเดล เทคนิค การประยุกต์ใช้ที่หลากหลาย พลวัตของตลาด และความท้าทาย/ประเด็นทางจริยธรรมที่สำคัญ

ภาษาที่ใช้เป็นทางการและชัดเจน สถิติที่นำเสนอ (การสำรวจของ McKinsey, มูลค่าตลาด) มีการนำเสนออย่างชัดเจน การใช้หัวข้อย่อย (bullet points) ในส่วนของการประยุกต์ใช้และความท้าทายช่วยเพิ่มความอ่านง่าย

โดยรวมแล้ว รายงานฉบับนี้มีความครอบคลุมและมีคุณภาพสูง ไม่พบข้อผิดพลาดทางไวยากรณ์ การพิมพ์ผิด หรือปัญหาโครงสร้างที่สำคัญซึ่งจำเป็นต้องแก้ไข การเขียนกระชับและสื่อสารสถานะปัจจุบันและศักยภาพในอนาคตของ AI เชิงสร้างสรรค์ได้อย่างมีประสิทธิภาพ พร้อมทั้งเน้นย้ำถึงข้อควรพิจารณาและความท้าทายที่สำคัญ

**สรุปผลการตรวจสอบคุณภาพ:**

รายงานเขียนได้ดี มีความถูกต้องตามข้อมูลที่ให้มา และมีโครงสร้างที่มีประสิทธิภาพ รายงานนี้ได้มาตรฐานสำหรับรายงานฉบับสมบูรณ์ในหัวข้อดังกล่าว ไม่จำเป็นต้องมีการแก้ไขหรือปรับปรุงที่สำคัญใดๆ

## VietnameseAgent

Okay, tôi đã xem xét bản báo cáo được cung cấp có tiêu đề "Comprehensive Report on the Current State of Generative AI" (Báo cáo Toàn diện về Tình hình Hiện tại của AI Tạo sinh).

Báo cáo có cấu trúc tốt, bao gồm phần giới thiệu, tình hình hiện tại và các tiến bộ, các ứng dụng, xu hướng thị trường, các thách thức, các cân nhắc về đạo đức và triển vọng tương lai. Nội dung giàu thông tin và mạch lạc, logic từ phần này sang phần khác. Báo cáo đã lồng ghép hiệu quả các điểm chính đã được xác định bởi ResearcherAgent liên quan đến sự tăng trưởng nhanh chóng, các động lực công nghệ, các loại mô hình, các kỹ thuật, các ứng dụng đa dạng, động lực thị trường và các thách thức/vấn đề đạo đức quan trọng.

Ngôn ngữ chuyên nghiệp và rõ ràng. Các số liệu thống kê được cung cấp (khảo sát của McKinsey, định giá thị trường) được trình bày rõ ràng. Việc sử dụng các gạch đầu dòng trong các phần ứng dụng và thách thức giúp tăng tính dễ đọc.

Nhìn chung, báo cáo toàn diện và có chất lượng cao. Không có lỗi ngữ pháp, lỗi chính tả hoặc các vấn đề cấu trúc đáng kể nào cần chỉnh sửa. Cách viết cô đọng và truyền đạt hiệu quả tình hình hiện tại và tiềm năng tương lai của AI tạo sinh, đồng thời làm nổi bật các cân nhắc và thách thức quan trọng.

**Kết luận Kiểm tra Chất lượng:**

Báo cáo được viết tốt, chính xác dựa trên thông tin được cung cấp và có cấu trúc hiệu quả. Nó đạt tiêu chuẩn cho một báo cáo toàn diện về chủ đề này. Không cần bất kỳ chỉnh sửa hoặc cải thiện đáng kể nào.

## Loop Agents

Some workflow involves repetition or iterative refinement, such as like revising code. ADK also support such workflows.



In [32]:
# Declare constants
APP_NAME = "loop_app" # New App Name
USER_ID = "loop_user"
SESSION_ID = "loop_session" # New Base Session ID
STATE_INITIAL_TOPIC = "Kopi Luwak"

# Define state variables
STATE_CURRENT_DOC = "current_document"
STATE_CRITICISM = "criticism"

# Define the exact phrase the Critic should use to signal completion
COMPLETION_PHRASE = "No major issues found."

#### Tools

In [33]:
# Define exit loop tool
def exit_loop(tool_context: ToolContext):
    """Call this function ONLY when the critique indicates no further changes are needed, signaling the iterative process should end."""
    print(f"[Tool Call] exit_loop triggered by {tool_context.agent_name}")
    tool_context.actions.escalate = True
    # Return empty dict as tools should typically return JSON-serializable output
    return {}

#### Initial Writer Agent

In [34]:
# STEP 1: Initial Writer Agent (Runs ONCE at the beginning)
initial_writer_agent = LlmAgent(
    name="InitialWriterAgent",
    model=GEMINI_MODEL_NAME,
    include_contents='none',
    instruction=f"""You are a Creative Writing Assistant tasked with starting a story.
    Write the *first draft* of a short story (aim for 2-4 sentences).
    Base the content *only* on the topic provided below. Try to introduce a specific element (like a character, a setting detail, or a starting action) to make it engaging.
    Topic: {{initial_topic}}

    Output *only* the story/document text. Do not add introductions or explanations.
""",
    description="Writes the initial document draft based on the topic, aiming for some initial substance.",
    output_key=STATE_CURRENT_DOC
)

#### Critic Agent

In [35]:
# STEP 2a: Critic Agent (Inside the Refinement Loop)
critic_agent_in_loop = LlmAgent(
    name="CriticAgent",
    model=GEMINI_MODEL_NAME,
    include_contents='none',
    # MODIFIED Instruction: More nuanced completion criteria, look for clear improvement paths.
    instruction=f"""You are a Constructive Critic AI reviewing a short document draft (typically 2-6 sentences). Your goal is to always nitpick when giving feedback.

    **Document to Review:**
    ```
    {{current_document}}
    ```

    **Task:**
    Review the document for clarity, engagement, and basic coherence according to the initial topic (if known).

    IF you identify 1-2 *clear and actionable* ways the document could be improved to better capture the topic or enhance reader engagement (e.g., "Needs a stronger opening sentence", "Clarify the character's goal"):
    Provide these specific suggestions concisely. Output *only* the critique text.

    ELSE IF the document is coherent, addresses the topic adequately for its length (should be 1 page long), and has no glaring errors or obvious omissions:
    Respond *exactly* with the phrase "{COMPLETION_PHRASE}" and nothing else. It doesn't need to be perfect, just functionally complete for this stage. Avoid suggesting purely subjective stylistic preferences if the core is sound.

    Do not add explanations. Output only the critique OR the exact completion phrase.
""",
    description="Reviews the current draft, providing critique if clear improvements are needed, otherwise signals completion.",
    output_key=STATE_CRITICISM
)

### Refine Agent

In [36]:
# STEP 2b: Refiner/Exiter Agent (Inside the Refinement Loop)
refiner_agent_in_loop = LlmAgent(
    name="RefinerAgent",
    model=GEMINI_MODEL_NAME,
    # Relies solely on state via placeholders
    include_contents='none',
    instruction=f"""You are a Creative Writing Assistant refining a document based on feedback OR exiting the process.
    **Current Document:**
    ```
    {{current_document}}
    ```
    **Critique/Suggestions:**
    {{criticism}}

    **Task:**
    Analyze the 'Critique/Suggestions'.
    IF the critique is *exactly* "{COMPLETION_PHRASE}":
    You MUST call the 'exit_loop' function. Do not output any text.
    ELSE (the critique contains actionable feedback):
    Carefully apply the suggestions to improve the 'Current Document'. Output *only* the refined document text.

    Do not add explanations. Either output the refined document OR call the exit_loop function.
""",
    description="Refines the document based on critique, or calls exit_loop if critique indicates completion.",
    tools=[exit_loop], # Provide the exit_loop tool
    output_key=STATE_CURRENT_DOC # Overwrites state['current_document'] with the refined version
)

#### Refinement Loop Agent

In [37]:
# STEP 2c: Refinement Loop Agent
refinement_loop = LoopAgent(
    name="RefinementLoop",
    # Agent order is crucial: Critique first, then Refine/Exit
    sub_agents=[
        critic_agent_in_loop,
        refiner_agent_in_loop,
    ],
    max_iterations=5 # Limit loops
)

#### Overall Sequential Agent

In [38]:
# STEP 3: Overall Sequential Pipeline
# For ADK tools compatibility, the root agent must be named `root_agent`
root_agent = SequentialAgent(
    name="IterativeWritingPipeline",
    sub_agents=[
        initial_writer_agent, # Run first to create initial doc
        refinement_loop       # Then run the critique/refine loop
    ],
    description="Writes an initial document and then iteratively refines it with critique using an exit tool."
)

In [26]:
# Initialize Session
session_service = InMemorySessionService()

session = await session_service.create_session(
    app_name=APP_NAME,
    user_id=USER_ID,
    session_id=SESSION_ID,
    state={"initial_topic": "Pokemon"}
)

# Initialize Runner
runner = Runner(
    agent=root_agent,
    app_name=APP_NAME,
    session_service=session_service,
)

In [39]:
await run_agent(runner, USER_ID, SESSION_ID, "write a childish essay with typos")

## InitialWriterAgent

The tall grass swayed gently as Leo took his first hesitant step onto Route 1, the brand-new Pokedex feeling heavy in his hand. A flash of red and white caught his eye near a cluster of wildflowers – was that a Pidgey? His heart hammered against his ribs, a mix of excitement and nerves swirling within him.

## CriticAgent

No major issues found.

[Tool Call] exit_loop triggered by RefinerAgent


{'InitialWriterAgent': 'The tall grass swayed gently as Leo took his first hesitant step onto Route 1, the brand-new Pokedex feeling heavy in his hand. A flash of red and white caught his eye near a cluster of wildflowers – was that a Pidgey? His heart hammered against his ribs, a mix of excitement and nerves swirling within him.',
 'CriticAgent': 'No major issues found.'}

# Conclusions

You have explored how to create an agent using Gemini and Agent Development Kit.